In [67]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
# 경로 변수 (고정)


trainfile = '../_data/train_03_997.csv'
old_trainfile = '../_data/train.csv'
kaggle_trainfile = '../_data/train_k.csv'

testfile = '../_data/test_clean.csv'
old_testfile = '../_data/test.csv'

train_171016 = '../_data/171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv'
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = '../_data/train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'

train_701778_all_vars = '../_data/train__701778__all_vars.csv'
train_700806_all_vars_cooks_dropped = '../_data/train_700806_dropped_cooks.csv'
train_700789_all_vars_cooks_dropped_circle_filter = '../_data/train_700789_dropped_cooks__circle_filter.csv'
train_700789_all_vars_cdcf_kmeans = '../_data/train_kmeans_30_added.csv'
train_700419_all_vars_cdcf150kmh_km60 = '../_data/train_kmeans_60_added_cooksd_circle_150kmh.csv'

train_700419_all_vars_4d_clust120 = '../_data/train_kmeans_4D_120.csv'
train_700419_all_vars_4d_clust120_speed = '../_data/train_kmeans_4D_120_clustspeed.csv'
train_700419_all_vars_4d_clust120_speed_glob = '../_data/train_kmeans_4D_120_clustspeed_globalized.csv'

train_700419 = '../_data/train_700419_recent.csv'
train_700419 = '../_data/train_700419_recent_2047.csv'
train_697355 = '../_data/train_697355_rsq_0722.csv'
train_697349 = '../_data/train_697349_rsq_0723.csv'
train_697284 = '../_data/train_697284_rsq_0725.csv'
train_696688 = '../_data/train_696688_rsq_0731.csv'

test_recent = '../_data/test_recent_2048.csv'

trains = [train_700419, train_697355, train_697349, train_697284, train_696688]

train_recent = trains[0]



In [4]:
pd.options.display.max_columns = 40

```
attempt k-means
```

In [5]:
# train recent
df_train = pd.read_csv(train_recent)
df_train.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,430,Saturday,23,19,59,22.972353,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454.0
1,0.0,False,0.0,430,Saturday,23,11,0,28.539090,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409.0
2,0.0,False,0.0,430,Saturday,23,38,63,25.137761,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081.0
3,0.0,False,0.0,430,Saturday,23,35,55,11.904820,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800.0
4,0.0,False,0.0,430,Saturday,23,14,79,13.197395,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151.0


In [6]:
# test recent
df_test = pd.read_csv(test_recent)
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,False,False,False,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,False,False,False,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,False,False,False,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,True,False,False,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,False,False,False,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN


In [7]:
# df of results
df_res = pd.read_csv('../_data/OLS_result_variables_unique_with_restr_171024_1507.csv')
df_res = df_res.tail()
df_res

,formula,desc,nobs,rsquared_adj,fvalue,f_pvalue,aic,bic,JB,JB-pval,skew,kurt,condition_number
8,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow)...,"log dur, hr^3, clust_spd with means, log time,...",700419.0,0.701208,45660.575994,0.0,757875.481676,758299.480734,3.446769e+06,0.0,0.516736,13.818326,106.364487
9,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow)...,"log dur, hr^3, new_clust_spd, log total_time, ...",697355.0,0.722529,51883.673856,0.0,680320.051067,680732.432863,1.051437e+06,0.0,0.608973,8.890892,107.983126
10,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow)...,"log dur, hr^3, new_clust_spd, log total_time, ...",697349.0,0.723376,52103.264707,0.0,678258.040967,678670.422453,9.006055e+05,0.0,0.577827,8.446077,107.996783
11,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow)...,"log dur, hr^3, new_clust_spd, log total_time, ...",697284.0,0.725532,52663.908834,0.0,672780.197744,673192.575874,7.351040e+05,0.0,0.526259,7.918734,108.034106
12,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow)...,"log dur, hr^3, new_clust_spd, log total_time, ...",696688.0,0.731000,54093.312934,0.0,649597.340000,650009.687347,3.894761e+05,0.0,0.382307,6.582222,108.378739


In [9]:
df_res.iloc[0, 2]

700419.0

In [49]:
df_res

,formula,desc,nobs,rsquared_adj,fvalue,f_pvalue,aic,bic,JB,JB-pval,skew,kurt,condition_number
8,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move),"log dur, hr^3, clust_spd with means, log time, didnt_move",700419.0,0.701208,45660.575994,0.0,757875.481676,758299.480734,3.446769e+06,0.0,0.516736,13.818326,106.364487
9,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp),"log dur, hr^3, new_clust_spd, log total_time, restr: dist=0 / dist<=100, dur>=900 / spd>100, dur<=20",697355.0,0.722529,51883.673856,0.0,680320.051067,680732.432863,1.051437e+06,0.0,0.608973,8.890892,107.983126
10,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp),"log dur, hr^3, new_clust_spd, log total_time, restr: dist=0 / dist<=50, dur>=600 / spd>100, dur<=20",697349.0,0.723376,52103.264707,0.0,678258.040967,678670.422453,9.006055e+05,0.0,0.577827,8.446077,107.996783
11,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp),"log dur, hr^3, new_clust_spd, log total_time, restr: dist=0 / dist<=50, dur>=300 / spd>100, dur<=20",697284.0,0.725532,52663.908834,0.0,672780.197744,673192.575874,7.351040e+05,0.0,0.526259,7.918734,108.034106
12,np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp),"log dur, hr^3, new_clust_spd, log total_time, restr: dist=0 / spd<0.3 / spd>100, dur<=20",696688.0,0.731000,54093.312934,0.0,649597.340000,650009.687347,3.894761e+05,0.0,0.382307,6.582222,108.378739


In [46]:
# for idx in range(1, 5):
#     df_res.iloc[idx, 0] = df_res.iloc[idx, 0][:-16]
#     print(df_res.iloc[idx, 0])

In [13]:
fmla = 'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)'

In [11]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)'

In [14]:
mod = sm.OLS.from_formula(fmla, data=df_train)
print('m')

m


In [15]:
res = mod.fit()

In [16]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(I(dur + 1))   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                 4.572e+04
Date:                Tue, 24 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:54:07   Log-Likelihood:            -3.7856e+05
No. Observations:              700419   AIC:                         7.572e+05
Df Residuals:                  700382   BIC:                         7.576e+05
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.4772      0.001   5138.522      0.000       6.475       6.480
C(is_hol)[T.True]                      -0.1241      0.004    -34.661      0.000      -0.131      -0.117
C(dow)[T.Monday]                       -0.0937      0.002    -49.051      0.000      -0.097      -0.090
C(dow)[T.Saturday]                     -0.1020      0.002    -57.011      0.000      -0.105      -0.098
C(dow)[T.Sunday]                       -0.2038      0.002   -107.309      0.000      -0.208      -0.200
C(dow)[T.Thursday]                      0.0074      0.002      4.100      0.000       0.004       0.011
C(dow)[T.Tuesday]                      -0.0293      0.002    -15.687      0.000      -0.033      -0.026
C(dow)[T.Wednesday]                    -0.0127      0.002     -6.924      0.000      -0.016      -0.009
C(p_LGA)[T.True]                        0.0338      0.003      9.810      0.000       0.027       0.040
C(d_JFK)[T.True]                        0.0503      0.007      7.250      0.000       0.037       0.064
C(d_LGA)[T.True]                       -0.0810      0.005    -15.896      0.000      -0.091      -0.071
C(didnt_move)[T.1.0]                    3.5620      0.009    395.542      0.000       3.544       3.580
C(dow)[Friday]:scale(hr)                0.3055      0.011     27.895      0.000       0.284       0.327
C(dow)[Monday]:scale(hr)                0.4976      0.014     35.157      0.000       0.470       0.525
C(dow)[Saturday]:scale(hr)             -0.4080      0.011    -37.339      0.000      -0.429      -0.387
C(dow)[Sunday]:scale(hr)               -0.5989      0.012    -50.185      0.000      -0.622      -0.576
C(dow)[Thursday]:scale(hr)              0.4798      0.012     38.768      0.000       0.456       0.504
C(dow)[Tuesday]:scale(hr)               0.5677      0.014     40.894      0.000       0.540       0.595
C(dow)[Wednesday]:scale(hr)             0.4922      0.013     37.611      0.000       0.467       0.518
C(dow)[Friday]:scale(I(hr ** 2))       -0.0445      0.028     -1.615      0.106      -0.098       0.009
C(dow)[Monday]:scale(I(hr ** 2))       -0.3231      0.034     -9.577      0.000      -0.389      -0.257
C(dow)[Saturday]:scale(I(hr ** 2))      1.4350      0.029     50.086      0.000       1.379       1.491
C(dow)[Sunday]:scale(I(hr ** 2))        1.9556      0.032     60.905      0.000       1.893       2.018
C(dow)[Thursday]:scale(I(hr ** 2))     -0.1620      0.030     -5.416      0.000      -0.221      -0.103
C(dow)[Tuesday]:scale(I(hr ** 2))      -0.3108      0.033     -9.513      0.000      -0.375      -0.247
C(dow)[Wednesday]:scale(I(hr ** 2))    -0.1404      0.031     -4.486      0.000      -0.202      -0.079
C(dow)[Friday]:scale(I(hr ** 3))       -0.2360      0.018    -13.26

In [40]:
def get_date_rowrange(df, d1, d2):
    top = df[(df['m_d'] >= d1)
                   &(df['m_d'] < d2)].iloc[0, :].name
    bot = df[(df['m_d'] >= d1)
                   &(df['m_d'] < d2)].iloc[-1, :].name
    return top, bot

def get_month_rowranges(df):
    month_ranges = dict()

    months = [101, 201, 301, 401, 501, 601]

    for idx, month in enumerate(months):
        try:
            month_ranges[idx+1] = get_date_rowrange(df, month, month+100)
        except:
            continue
    return month_ranges

print(get_date_rowrange(df_train, 115, 211))
get_month_rowranges(df_train)


(574696, 560225)


{1: (687677, 560225), 2: (389557, 506738), 3: (279855, 277068)}

In [52]:
df_train.shape

(200000, 31)

In [50]:
df_train[(df_train['m_d'] >= 401)
                   &(df_train['m_d'] < 501)]

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur


In [48]:
get_date_rowrange(df_train, 401, 501)[1] - get_date_rowrange(df_train, 101, 401)[0] + 1

IndexError: single positional indexer is out-of-bounds

In [ ]:
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)

In [44]:
import random

def train_test_date_samples(X, y, train_size=None,\
                            test_size=None,\
                            train_range=(101, 401),
                            test_range=(401, 501)):
    train_start, train_end = get_date_rowrange(X, *train_range)
    test_start, test_end = get_date_rowrange(X, *test_range)

    train_cnt = train_end - train_start + 1
    test_cnt = test_end - test_start + 1
    
    if not train_size or train_size > train_cnt:
        train_size = train_cnt
    if not test_size or test_size > test_cnt:
        test_size = test_cnt
    
    train_indices = random.sample(range(train_start, train_end+1),\
                                  k=train_size)
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)
    
    print("test / (train + test) = {}".format(test_size / (train_size + test_size)))
        
    return X.loc[train_indices, :], X.loc[test_indices, :],\
           y.loc[train_indices], y.loc[test_indices]

    

X, y = df_train.iloc[:, :-1], df_train.iloc[:, -1]
    
a, b, c, d = train_test_date_samples(X, y)


IndexError: single positional indexer is out-of-bounds

In [64]:
df_train = pd.read_csv('../_data/train_700419_recent_2047.csv')
df_train.shape

(700419, 31)

In [68]:
df_trainz = df_train.copy()

In [69]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

from datetime import datetime

t00 = datetime.now()
print(t00)

fmlas = [fmla]
df_trains = [df_trainz]

for idx in range(len(df_trains)):
    t0 = datetime.now()
    print(t0)
    
    fmla = fmlas[idx]
    df = df_trains[idx]
    scores = np.zeros(1)
    
    print("Train has {} rows".format(df.shape[0]))
    print("Test has {} rows".format(df.shape[0]))
    print("formula = {}\n".format(fmla))
    
    for i in range(2):
        X_train, X_test, y_train, y_test = train_test_date_samples(df.iloc[:, :-1].copy(), df.iloc[:, -1].copy(),\
                                                                    train_size = 200000,\
                                                                    test_size = 50000)
        df_train__ = pd.concat([X_train, y_train], axis=1)

        print("trial {}".format(i+1))
        model_train = sm.OLS.from_formula(fmla, data=df_train__)
        res_train = model_train.fit()

        y_pred = res_train.predict(X_test)
        
        y_pred = np.exp(y_pred)

        try:
            scores[i] = mean_squared_log_error(y_test, y_pred)
            print(pd.concat([y_test, y_pred], axis=1).head(5))
        except:
            print("except")
            print(df.shape)
            print(y_test[y_test<=0].shape)
            print(y_pred[y_pred<=0].shape)
            break

    scores = np.sqrt(scores)
    print(scores)
    
    t1 = datetime.now()
    print(t1)
    print(t1 - t0)
    print()
    print()

    
t2 = datetime.now()
print(t2)
print(t2 - t00)

2017-10-24 21:11:15.007711
2017-10-24 21:11:15.008688
Train has 700419 rows
Test has 700419 rows
formula = np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)

test / (train + test) = 0.2
trial 1
           dur            0
88933    787.0   890.580657
103732   657.0   214.226008
141881  2013.0  1348.623338
92113   1305.0   718.909701
52231    619.0   153.844245
test / (train + test) = 0.2
trial 2
except
(200000, 31)
(0,)
(0,)
[ 0.42489003]
2017-10-24 21:11:41.296093
0:00:26.287405


2017-10-24 21:11:41.296093
0:00:26.288382


In [71]:
df_train.shape

(700419, 31)

In [72]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)'

In [73]:
mod = sm.OLS.from_formula(fmla, data=df_train)
res = mod.fit()

In [74]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(I(dur + 1))   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                 4.572e+04
Date:                Tue, 24 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:14:58   Log-Likelihood:            -3.7856e+05
No. Observations:              700419   AIC:                         7.572e+05
Df Residuals:                  700382   BIC:                         7.576e+05
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.4772      0.001   5138.522      0.000       6.475       6.480
C(is_hol)[T.True]                      -0.1241      0.004    -34.661      0.000      -0.131      -0.117
C(dow)[T.Monday]                       -0.0937      0.002    -49.051      0.000      -0.097      -0.090
C(dow)[T.Saturday]                     -0.1020      0.002    -57.011      0.000      -0.105      -0.098
C(dow)[T.Sunday]                       -0.2038      0.002   -107.309      0.000      -0.208      -0.200
C(dow)[T.Thursday]                      0.0074      0.002      4.100      0.000       0.004       0.011
C(dow)[T.Tuesday]                      -0.0293      0.002    -15.687      0.000      -0.033      -0.026
C(dow)[T.Wednesday]                    -0.0127      0.002     -6.924      0.000      -0.016      -0.009
C(p_LGA)[T.True]                        0.0338      0.003      9.810      0.000       0.027       0.040
C(d_JFK)[T.True]                        0.0503      0.007      7.250      0.000       0.037       0.064
C(d_LGA)[T.True]                       -0.0810      0.005    -15.896      0.000      -0.091      -0.071
C(didnt_move)[T.1.0]                    3.5620      0.009    395.542      0.000       3.544       3.580
C(dow)[Friday]:scale(hr)                0.3055      0.011     27.895      0.000       0.284       0.327
C(dow)[Monday]:scale(hr)                0.4976      0.014     35.157      0.000       0.470       0.525
C(dow)[Saturday]:scale(hr)             -0.4080      0.011    -37.339      0.000      -0.429      -0.387
C(dow)[Sunday]:scale(hr)               -0.5989      0.012    -50.185      0.000      -0.622      -0.576
C(dow)[Thursday]:scale(hr)              0.4798      0.012     38.768      0.000       0.456       0.504
C(dow)[Tuesday]:scale(hr)               0.5677      0.014     40.894      0.000       0.540       0.595
C(dow)[Wednesday]:scale(hr)             0.4922      0.013     37.611      0.000       0.467       0.518
C(dow)[Friday]:scale(I(hr ** 2))       -0.0445      0.028     -1.615      0.106      -0.098       0.009
C(dow)[Monday]:scale(I(hr ** 2))       -0.3231      0.034     -9.577      0.000      -0.389      -0.257
C(dow)[Saturday]:scale(I(hr ** 2))      1.4350      0.029     50.086      0.000       1.379       1.491
C(dow)[Sunday]:scale(I(hr ** 2))        1.9556      0.032     60.905      0.000       1.893       2.018
C(dow)[Thursday]:scale(I(hr ** 2))     -0.1620      0.030     -5.416      0.000      -0.221      -0.103
C(dow)[Tuesday]:scale(I(hr ** 2))      -0.3108      0.033     -9.513      0.000      -0.375      -0.247
C(dow)[Wednesday]:scale(I(hr ** 2))    -0.1404      0.031     -4.486      0.000      -0.202      -0.079
C(dow)[Friday]:scale(I(hr ** 3))       -0.2360      0.018    -13.26

In [77]:
del df_test['dur']

In [78]:
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,False,False,False,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,False,False,False,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,False,False,False,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,True,False,False,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,False,False,False,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [82]:
test_pred_log = res.predict(df_test)

In [83]:
test_pred = np.exp(test_pred_log)

In [84]:
test_pred.head()

0    1007.790371
1     408.912734
2    1850.600478
3    1382.675226
4     748.497554
dtype: float64

In [85]:
df_test_orig = pd.read_csv('../_data/test.csv')
df_test_orig.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2332349,2,6/30/16 23:59,5,-73.998291,40.722618,-73.971779,40.762470,N
1,id2013516,1,6/30/16 23:59,2,-73.955109,40.689564,-73.978203,40.685802,N
2,id3719493,2,6/30/16 23:59,1,-73.978416,40.791576,-73.973160,40.675968,N
3,id0376262,1,6/30/16 23:59,2,-73.873093,40.774097,-73.926704,40.856739,N
4,id2105228,1,6/30/16 23:58,1,-73.979416,40.755211,-74.003548,40.738224,N


In [86]:
df_test_orig.shape

(346797, 9)

In [87]:
test_pred.shape

(346797,)

In [88]:
df_final = pd.concat([df_test_orig['id'], test_pred], axis=1, names=['id', 'trip_duration'])

In [89]:
df_final

,id,0
0,id2332349,1007.790371
1,id2013516,408.912734
2,id3719493,1850.600478
3,id0376262,1382.675226
4,id2105228,748.497554
5,id1276841,571.839664
6,id3016059,409.326217
7,id3952659,674.618586
8,id1255468,446.025276
9,id3049611,407.534003


In [92]:
df_final.to_csv('../_data/submission_2120.csv', index=False)

In [93]:
df_final_backup = df_final

replace 0 dist values

In [96]:
df_raw = pd.read_csv('../_data/train__701778__all_vars.csv')
df_raw.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [105]:
avg_zero = df_raw.loc[df_raw['total_dist'] == 0, 'dur'].mean()
avg_zero

503.51274886482713

In [107]:
df_test['dur'] = test_pred

In [108]:
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,False,False,False,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1007.790371
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,False,False,False,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,408.912734
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,False,False,False,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1850.600478
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,True,False,False,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1382.675226
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,False,False,False,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,748.497554


In [ ]:
'p_lng', 'p_lat', 'd_lng', 'd_lat'

In [110]:
df_test_small = df_test[['id', 'didnt_move', 'p_lng', 'p_lat', 'd_lng', 'd_lat', 'total_dist', 'dur']].copy()

In [114]:
df_test_small.loc[df_test_small['didnt_move'] == 1, 'dur'] = avg_zero

In [118]:
df_test_small.insert(0, 'id', df_test_orig['id'])
df_test_small.head()

,id,didnt_move,p_lng,p_lat,d_lng,d_lat,total_dist,dur
0,id2332349,0.0,-73.998291,40.722618,-73.971779,40.762470,5280.3,1007.790371
1,id2013516,0.0,-73.955109,40.689564,-73.978203,40.685802,2573.5,408.912734
2,id3719493,0.0,-73.978416,40.791576,-73.973160,40.675968,15048.8,1850.600478
3,id0376262,0.0,-73.873093,40.774097,-73.926704,40.856739,16818.3,1382.675226
4,id2105228,0.0,-73.979416,40.755211,-74.003548,40.738224,3536.7,748.497554


In [121]:
df_final_2 = df_test_small.loc[:, ['id', 'dur']]

In [126]:
df_final_2.columns = ['id', 'trip_duration']
df_final_2.to_csv('../_data/submission_2131.txt', index=False)

In [115]:
df_test_small.loc[df_test_small['total_dist'] == 0.0, :]

,didnt_move,p_lng,p_lat,d_lng,d_lat,total_dist,dur
394,1.0,-73.892860,40.765896,-73.892860,40.765896,0.0,503.512749
490,1.0,-74.325340,40.723003,-74.325340,40.723003,0.0,503.512749
763,1.0,-74.030060,40.749531,-74.030060,40.749531,0.0,503.512749
931,1.0,-74.040298,40.621075,-74.040298,40.621075,0.0,503.512749
1518,1.0,-74.051002,40.604469,-74.051002,40.604469,0.0,503.512749
1917,1.0,-73.794342,40.657082,-73.794342,40.657082,0.0,503.512749
2231,1.0,-74.005089,40.731529,-74.005089,40.731529,0.0,503.512749
2442,1.0,-73.776337,40.645329,-73.776337,40.645329,0.0,503.512749
2487,1.0,-73.945427,40.751575,-73.945427,40.751575,0.0,503.512749
2526,1.0,-73.964729,40.771557,-73.964729,40.771557,0.0,503.512749


uses train_test_date_sample2

In [29]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

from datetime import datetime

t00 = datetime.now()
print(t00)

df0 = df_trains[0]
scores_all = list()

for idx in range(len(df_trains)):
    t0 = datetime.now()
    print(t0)
    
    fmla = fmlas[idx]
    df = df_trains[idx]
    scores = np.zeros(1)
    
    print("Train has {} rows".format(df.shape[0]))
    print("Test has {} rows".format(df0.shape[0]))
    print("formula = {}\n".format(fmla))
    
    for i in range(1):
        X_train, X_test, y_train, y_test = train_test_date_samples2(df.iloc[:, :-1], df.iloc[:, -1],\
                                                                    df0.iloc[:, :-1], df0.iloc[:, -1],\
                                                                    train_size = 200000,\
                                                                    test_size = 50000)
        df_train = pd.concat([X_train, y_train], axis=1)

        print("trial {}".format(i+1))
        model_train = sm.OLS.from_formula(fmla, data=df_train)
        res_train = model_train.fit()

        y_pred = res_train.predict(X_test)
        
        y_pred = np.exp(y_pred)

        try:
            scores[i] = mean_squared_log_error(y_test, y_pred)
            print(pd.concat([y_test, y_pred], axis=1).head(5))
        except:
            print("except")
            print(y_test[y_test<=0].shape)
            print(y_pred[y_pred<=0].shape)
            break

    scores = np.sqrt(scores)
    print(scores)
    scores_all.append(scores)
    
    t1 = datetime.now()
    print(t1)
    print(t1 - t0)
    print()
    print()

    
t2 = datetime.now()
print(t2)
print(t2 - t00)

2017-10-24 20:57:08.980862
2017-10-24 20:57:08.981864
Train has 700419 rows


NameError: name 'df0' is not defined

In [151]:
scores_all

array([ 0.4229259,  0.4229259,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ])

In [36]:
res_list = list()

for idx in len(df_trains):
    print(idx, end=' ')
    mod = sm.OLS.from_formula(df_res.iloc[idx, 0], data=df_trains[idx])
    res_list.append(mod)

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)'

In [181]:
df_testz.to_csv('../_data/test_recent_2048.csv', index=False)

In [35]:
import random

def train_test_date_samples2(X, y, X0, y0, train_size=None,\
                            test_size=None,\
                            train_range=(101, 401),
                            test_range=(401, 501)):
    train_start, train_end = get_date_rowrange(X, *train_range)
    test_start, test_end = get_date_rowrange(X, *test_range)

    train_cnt = train_end - train_start + 1
    test_cnt = test_end - test_start + 1
    
    if not train_size or train_size > train_cnt:
        train_size = train_cnt
    if not test_size or test_size > test_cnt:
        test_size = test_cnt
    
    train_indices = random.sample(range(train_start, train_end+1),\
                                  k=train_size)
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)
    
    print("test / (train + test) = {}".format(test_size / (train_size + test_size)))
        
    return X.loc[train_indices, :], X0.loc[test_indices, :],\
           y.loc[train_indices], y0.loc[test_indices]

    

X, y = df_train.iloc[:, :-1], df_train.iloc[:, -1]
    
a, b, c, d = train_test_date_samples2(X, y, X, y)


IndexError: single positional indexer is out-of-bounds